<a href="https://colab.research.google.com/github/Greqorian/Ebay_kleineanzeige_img_scraper/blob/main/Kleineanzeige_Images_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# package imports
#basics
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json # json files

#scrapping
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import time


In [ ]:
# Set up the headers of the request (User-Agent) as Google boot. Otherwise Ebay-Kleinanzeigen blocks requests.
headers = {
      'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)',
  }

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Eine Funktion um eine Text-Datei zu beschreiben.
def writeFile(response, name ,fileType):
    with open(name + fileType, "a", encoding="utf8") as f:
        f.write(response)

In [ ]:
# gets BeautifulSoup instance from url request
def getSoupFromUtl(url):
  
  # saves HTML of the webpage to the variable
  response = requests.get(url, headers=headers)

  # saves the content of the page
  page = response.content

  # creates an instance of BeautifulSoup with webpage content
  soup = BeautifulSoup(page, "html.parser")

  return soup

In [ ]:
# returns the list ofi items url adress pages from kleineanzeige search
def getItemsUlrsList(itemName):

  # creates a query of the search in ebay kleineanzeige
  query = "Ikea-" + itemName

  # add the Query to Ebay-Kleinanzeigen URL
  URL = "https://www.ebay-kleinanzeigen.de/s-" + query + "/k0"

  # gets BeautifulSoup instance from url request
  soup = getSoupFromUtl(URL)

  # selects a div element with id
  srchRsltsContent = soup.find("div", id="srchrslt-content")

  # selects all links from selected div
  srchRslts = srchRsltsContent.find_all('a', href=True)

  itemPagesUrls = []

  # save a list of subpages URLs
  for a in srchRslts:
      
      if '/s-anzeige' in a['href']:
        itemPagesUrls.append('https://www.ebay-kleinanzeigen.de' + a['href'])

  return itemPagesUrls


In [ ]:
# returns a list of sources for all item images
def getItemsImagesSrcList(itemPagesUrls):

  imagesSrcList = []

  for URL in itemPagesUrls:
    
    # gets BeautifulSoup instance from url request
    soupForSubPage = getSoupFromUtl(URL)

    srchRslts2 = soupForSubPage.find_all("div", {"class": "galleryimage-element"})

    # search for images adresses 
    for tag in srchRslts2:
      image = tag.find("img")
      if hasattr(image, 'src'):
        imagesSrcList.append(image['src'])

  return imagesSrcList


In [ ]:
# Opening JSON file
f = open('/content/drive/MyDrive/DATA/furnitureImages/1000-furniture/ikeaRangeList.json', "r")
# a dictionary
ikeaRangeListJson = json.load(f)

print(len(ikeaRangeListJson))


1000


In [ ]:
print(ikeaRangeListJson)

[{'id': '60510640', 'name': 'JÄRVFJÄLLET', 'title': 'JÄRVFJÄLLET Office chair with armrests', 'url': 'https://www.ikea.com/us/en/p/jaervfjaellet-office-chair-with-armrests-60510640/', 'currency': 'USD', 'price': '279', 'category': 'Furniture', 'group': 'Chairs', 'subgroup': 'Desk chairs'}, {'id': '49324512', 'name': 'ÄPPLARÖ', 'title': 'ÄPPLARÖ Table and 4 folding chairs, outdoor - brown stained/Frösön/Duvholmen dark gray', 'url': 'https://www.ikea.com/us/en/p/aepplaroe-table-and-4-folding-chairs-outdoor-brown-stained-froesoen-duvholmen-dark-gray-s49324512/', 'currency': 'USD', 'price': '268.96', 'category': 'Furniture', 'group': 'Patio furniture', 'subgroup': 'Outdoor dining sets'}, {'id': '19331892', 'name': 'LOBERGET', 'title': 'LOBERGET / BLYSKÄR Swivel chair with pad - white/blue', 'url': 'https://www.ikea.com/us/en/p/loberget-blyskaer-swivel-chair-with-pad-white-blue-s19331892/', 'currency': 'USD', 'price': '34.99', 'category': 'Furniture', 'group': 'Chairs', 'subgroup': 'Desk ch

In [ ]:
# Opening JSON file
# f = open('/content/drive/MyDrive/DATA/furnitureImages/5-first-furnitureImages/ebayItemsList.json', "r")
# itemsList = json.load(f)

In [ ]:
print(itemsList)

[{'id': '60510640', 'name': 'JÄRVFJÄLLET', 'urls': ['https://www.ebay-kleinanzeigen.de/s-anzeige/jaervfjaellet-drehstuhl-von-ikea-grau-/1986272780-93-3113', 'https://www.ebay-kleinanzeigen.de/s-anzeige/jaervfjaellet-drehstuhl-von-ikea-grau-/1986272780-93-3113', 'https://www.ebay-kleinanzeigen.de/s-anzeige/schreibtischstuhl-ikea-jaervfjaellet/1985942902-93-1442', 'https://www.ebay-kleinanzeigen.de/s-anzeige/schreibtischstuhl-ikea-jaervfjaellet/1985942902-93-1442', 'https://www.ebay-kleinanzeigen.de/s-anzeige/ikea-drehstuhl-schreibtischstuhl-jaervfjallet-neu/1985921702-93-4300', 'https://www.ebay-kleinanzeigen.de/s-anzeige/ikea-drehstuhl-schreibtischstuhl-jaervfjallet-neu/1985921702-93-4300', 'https://www.ebay-kleinanzeigen.de/s-anzeige/drehstuhl-ikea-jaervfjaellet-/1985105009-93-8126', 'https://www.ebay-kleinanzeigen.de/s-anzeige/drehstuhl-ikea-jaervfjaellet-/1985105009-93-8126', 'https://www.ebay-kleinanzeigen.de/s-anzeige/ikea-buerostuhl-jaervfjaellet-dunkelgrau-weiss-wie-neu/19847901

In [ ]:
itemsList = []

for item in ikeaRangeListJson:
  itemsList.append({'id': item['id'], 'name': item['name'], 'urls': getItemsUlrsList(item['name']), 'imgSrcs': []})




In [ ]:
print(itemsList) 

In [ ]:
for item in itemsList:
  item['imgSrcs'] = getItemsImagesSrcList(item['urls'])
  print(item['name'])
  print(item['urls'])
  print(len(item['imgSrcs']))
  print(item['imgSrcs'])
  time.sleep(10)

print(itemsList)

JÄRVFJÄLLET
['https://www.ebay-kleinanzeigen.de/s-anzeige/ikea-jaervfjaellet-buerostuhl/2043278084-93-4900', 'https://www.ebay-kleinanzeigen.de/s-anzeige/ikea-jaervfjaellet-buerostuhl/2043278084-93-4900', 'https://www.ebay-kleinanzeigen.de/s-anzeige/ikea-jaervfjaellet-schreibtischstuhl-wie-neu/2042779028-93-40', 'https://www.ebay-kleinanzeigen.de/s-anzeige/ikea-jaervfjaellet-schreibtischstuhl-wie-neu/2042779028-93-40', 'https://www.ebay-kleinanzeigen.de/s-anzeige/ikea-buerostuhl-jaervfjaellet/2042256023-93-9618', 'https://www.ebay-kleinanzeigen.de/s-anzeige/ikea-buerostuhl-jaervfjaellet/2042256023-93-9618', 'https://www.ebay-kleinanzeigen.de/s-anzeige/ikea-jaervfjaellet-buerostuhl-neu-vor-einer-woche-gekauft/2041188255-93-8869', 'https://www.ebay-kleinanzeigen.de/s-anzeige/ikea-jaervfjaellet-buerostuhl-neu-vor-einer-woche-gekauft/2041188255-93-8869', 'https://www.ebay-kleinanzeigen.de/s-anzeige/buerostuhl-von-ikea-jaervfjaellet/2040306239-93-1896', 'https://www.ebay-kleinanzeigen.de/s-

In [1]:
print(len(itemsList))

NameError: ignored

In [ ]:
os.chdir('/content/drive/MyDrive/DATA/furnitureImages/100-furniture')
!pwd

/content/drive/MyDrive/DATA/furnitureImages/100-furniture


Save the array of IKEA Products to JSON file

In [ ]:
# save the itemslist to JSON file
with open('ebayItemsList.json', 'w', encoding='utf-8') as outfile:
    json.dump(itemsList, outfile, ensure_ascii=False)

In [ ]:
os.chdir('/content/drive/MyDrive/DATA/furnitureImages/100-furniture/train')
!pwd

/content/drive/MyDrive/DATA/furnitureImages/100-furniture/train


In [ ]:
ebayImagesList = []

for item in itemsList:

  name = item['name'].replace('/', '_')
  id = item['id']

  for index, src in enumerate(item['imgSrcs']):
    fileName = str(index) + '_' + id + '_' + name + '_' + 'ebay' +'.jpg'
    ebayImagesList.append({'title': fileName, 'name':name})

    with open(fileName, 'wb') as f:
      im = requests.get(src, headers)
      f.write(im.content)
      f.close()
      print('Writing: ', fileName)
print(ebayImagesList)

In [ ]:
os.chdir('/content/drive/MyDrive/DATA/furnitureImages/100-furniture/')
!pwd
# save the ebayImagesList to JSON file
with open('ebayImagesList.json', 'w', encoding='utf-8') as outfile:
    json.dump(ebayImagesList, outfile, ensure_ascii=False)

/content/drive/MyDrive/DATA/furnitureImages/100-furniture


In [ ]:
print(len(ebayImagesList))

3828
